<h2>Опечатки<h2>

<h4>Метод берет список слов с ошибками и предлагает правильное слово для каждого неправильного слова. Он пытается найти в списке правильных вариантов написания слово с наименьшим расстоянием и той же начальной буквой, что и слово с ошибкой. Затем он возвращает слово, которое соответствует заданным критериям. Методы можно различать на основе меры расстояния, которую они используют для поиска ближайшего слова. В качестве словаря правильных слов используется пакет «words» от nltk.<h4>

<h4>Импортируем библиотеки<h4>

In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.metrics.distance import jaccard_distance
from nltk.util import ngrams
nltk.download('words')
from nltk.corpus import words
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
from collections import Counter
from model_selction import model_selection_word_count, model_selection_word_exist, model_selection_tfidf

[nltk_data] Downloading package words to /home/val/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /home/val/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
%ls data

p00_tweets.zip*         processedNeutral.csv*
processedNegative.csv*  processedPositive.csv*


<h4>В качестве примера рассмотрим содержимое файла 'processedNegative.csv' после применения метода<h4>

In [2]:
neg_df = pd.read_csv('data/processedNegative.csv').T.reset_index()
neg_text = " ".join([tweet[0] for tweet in neg_df.values.tolist()])
neg_tokens = word_tokenize(neg_text)
correct_words = words.words()
neg_spell = list()
for word in neg_tokens:
    if len(word) > 1:
        temp = [(jaccard_distance(set(ngrams(word, 2)),
                                set(ngrams(w, 2))),w)
                for w in correct_words if w[0]==word[0]]
        try:
            neg_spell.append(sorted(temp, key = lambda val:val[0])[0][1])
        except IndexError:
            neg_spell.append(word)
    else:
        neg_spell.append(word)
neg_spell = [word for word in neg_spell if not word in stopwords.words('english')]
neg_spell

['Ho',
 'unhappy',
 'dogs',
 'like',
 'though',
 'talking',
 'driver',
 'I',
 "'m",
 'going',
 'said',
 "'d",
 'love',
 'go',
 'Newar',
 'Yorker',
 'since',
 'Triumph',
 "'s",
 'probably',
 'Doeg',
 'anybody',
 'know',
 'Rand',
 "'s",
 'likely',
 'fall',
 'dollar',
 '?',
 'I',
 'got',
 'money',
 'I',
 'need',
 'change',
 'R',
 'keep',
 'getting',
 'strong',
 'unhappy',
 'I',
 'miss',
 'going',
 'gig',
 'Liverpudlian',
 'unhappy',
 'Theresa',
 'new',
 'Ricciales',
 'tonight',
 '?',
 'unhappy',
 "'s",
 'A',
 '*',
 'dye',
 'guy',
 'pop',
 'Asian',
 'translator',
 "'ll",
 'prob',
 'go',
 'around',
 'Aus',
 'unhappy',
 'Whig',
 "'s",
 'chair',
 "'re",
 'sitting',
 '?',
 'Isis',
 'I',
 'find',
 '.',
 'Everyman',
 'know',
 '.',
 'Yomud',
 "'ve",
 'shamed',
 'pu',
 'n',
 'like',
 'jittery',
 'caffeine',
 'make',
 'sad',
 'Mya',
 'area',
 "'s",
 'list',
 'unhappy',
 'think',
 'I',
 "'ll",
 'go',
 'Libby',
 'anyway',
 'I',
 'want',
 'fun',
 'plan',
 'weekend',
 'unhappy',
 'Wend',
 'notice',
 '.

<h4>Функция, которая создасть набор данных для обучения моделей<h4>

In [3]:
def spell_file_to_df(file_name):
    neg_fn, neut_fn, pos_fn = file_name

    neg_df = pd.read_csv(neg_fn).T.reset_index()
    neut_df = pd.read_csv(neut_fn).T.reset_index()
    pos_df = pd.read_csv(pos_fn).T.reset_index()
    
    neg_text = " ".join([tweet[0] for tweet in neg_df.values.tolist()])
    neut_text = " ".join([tweet[0] for tweet in neut_df.values.tolist()])
    pos_text = " ".join([tweet[0] for tweet in pos_df.values.tolist()])

    correct_words = words.words()
    
    neg_tokens = word_tokenize(neg_text)
    neg_spell = list()
    for word in neg_tokens:
        if len(word) > 1:
            temp = [(jaccard_distance(set(ngrams(word, 2)),
                                    set(ngrams(w, 2))),w)
                    for w in correct_words if w[0]==word[0]]
            try:
                neg_spell.append(sorted(temp, key = lambda val:val[0])[0][1])
            except IndexError:
                neg_spell.append(word)
        else:
            neg_spell.append(word)
    neg_spell = [word for word in neg_spell if not word in stopwords.words('english')]

    neut_tokens = word_tokenize(neut_text)
    neut_spell = list()
    for word in neut_tokens:
        if len(word) > 1:
            temp = [(jaccard_distance(set(ngrams(word, 2)),
                                    set(ngrams(w, 2))),w)
                    for w in correct_words if w[0]==word[0]]
            try:
                neut_spell.append(sorted(temp, key = lambda val:val[0])[0][1])
            except IndexError:
                neut_spell.append(word)
        else:
            neut_spell.append(word)
    neut_spell = [word for word in neut_spell if not word in stopwords.words('english')]

    pos_tokens = word_tokenize(pos_text)
    pos_spell = list()
    for word in pos_tokens:
        if len(word) > 1:
            temp = [(jaccard_distance(set(ngrams(word, 2)),
                                    set(ngrams(w, 2))),w)
                    for w in correct_words if w[0]==word[0]]
            try:
                pos_spell.append(sorted(temp, key = lambda val:val[0])[0][1])
            except IndexError:
                pos_spell.append(word)
        else:
            pos_spell.append(word)
    pos_spell = [word for word in pos_spell if not word in stopwords.words('english')]

    neg_words = Counter(neg_spell)
    neut_words = Counter(neut_spell)
    pos_words = Counter(pos_spell)
    
    unic_words = list(set(neg_words.keys()) | set(neut_words.keys()) | set(pos_words.keys()))

    neg_exist_index = 0
    neut_exist_index = 1
    pos_exist_index = 2
    neg_count_index = 3
    neut_count_index = 4
    pos_count_index = 5
    word_count_index = 6
    neg_tfidf_index = 7
    neut_tfidf_index = 8
    pos_tfidf_index = 9

    df = np.zeros((len(unic_words), 10))
    for i, word in enumerate(unic_words):
        if word in neg_words.keys():
            df[i,neg_exist_index] = 1
            df[i,neg_count_index] = neg_words[word]
        if word in neut_words.keys():
            df[i,neut_exist_index] = 1
            df[i,neut_count_index] = neut_words[word]
        if word in pos_words.keys():
            df[i,pos_exist_index] = 1
            df[i,pos_count_index] = pos_words[word]

    df[:,word_count_index] = df[:,neg_count_index] + df[:,neut_count_index] + df[:,pos_count_index]
    df[:,neg_tfidf_index] = df[:,neg_count_index] / df[:,word_count_index]
    df[:,neut_tfidf_index] = df[:,neut_count_index] / df[:,word_count_index]
    df[:,pos_tfidf_index] = df[:,pos_count_index] / df[:,word_count_index]

    spell_df = pd.DataFrame(df, columns=[
        'Negative', 'Neutral', 'Positive',
        'Negative counts', 'Neutral counts', 'Positive counts', 'Word counts',
        'Negative TFIDF', 'Neutral TFIDF', 'Positive TFIDF'])
    spell_df["word"] = unic_words
    return spell_df

<h4>Узнаем, как называются остальные файлы, содержащие исходный набор данных<h4>

In [4]:
%ls data

p00_tweets.zip*         processedNeutral.csv*
processedNegative.csv*  processedPositive.csv*


<h4>Создадим набор данных для обучения<h4>

In [4]:
file_names = ('data/processedNegative.csv', 'data/processedNeutral.csv', 'data/processedPositive.csv')
misspel_df = spell_file_to_df(file_names)
misspel_df

,Negative,Neutral,Positive,Negative counts,Neutral counts,Positive counts,Word counts,Negative TFIDF,Neutral TFIDF,Positive TFIDF,word
0,1.0,0.0,0.0,4.0,0.0,0.0,4.0,1.0,0.0,0.0,un
1,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,Comitium
2,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,storied
3,1.0,0.0,0.0,3.0,0.0,0.0,3.0,1.0,0.0,0.0,Terrance
4,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,safe
...,...,...,...,...,...,...,...,...,...,...,...
5863,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,restrain
5864,1.0,0.0,0.0,2.0,0.0,0.0,2.0,1.0,0.0,0.0,arse
5865,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,Bontok
5866,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,longe


<h4>Сохраним полученный набор данных, чтобы уменьшить ожидание при демонтсрации<h4>

In [5]:
misspel_df.to_csv('data/misspel.csv')

In [6]:
misspel_df = pd.read_csv('data/misspel.csv')
unic_words = misspel_df.word.tolist()

In [7]:
unic_words

['un',
 'Comitium',
 'storied',
 'Terrance',
 'safe',
 'needer',
 'Ike',
 'Chane',
 'nil',
 'param',
 'admit',
 'beginning',
 'haw',
 'mastermind',
 'decided',
 'whole',
 'accidental',
 'P',
 'free',
 'legitimate',
 'naked',
 'Sal',
 'Bat',
 'formerly',
 'watching',
 '900',
 'stipend',
 'horizon',
 'presumably',
 'Gamelion',
 'lorry',
 'information',
 'wipe',
 'anniversary',
 'seedage',
 'smoked',
 'voice',
 'prove',
 'Rivina',
 'tane',
 'Pianola',
 'gun',
 'immigrant',
 '.41',
 'spat',
 'regulator',
 'estrange',
 'wondering',
 'tamperer',
 'wont',
 'chilling',
 '3.41',
 'stunner',
 'stun',
 'aimer',
 'mobbish',
 'quite',
 'display',
 'Albright',
 'clarify',
 'hungry',
 'Umatilla',
 'wifie',
 'wave',
 'Diego',
 'incident',
 'Yun',
 'Somali',
 'discretionary',
 'Rajendra',
 'quickener',
 'Mariana',
 'embarrass',
 '9',
 'Trichina',
 'vale',
 'revive',
 'Sho',
 "'A7a",
 'Xyleborus',
 'medal',
 'Equus',
 'waffle',
 'forward',
 'Sydneyite',
 'spoil',
 'sima',
 'J',
 'snow',
 'mull',
 'happe

<h4>Узнаем полученную точность модели<h4>

In [9]:
word_exist_accuracy_score = model_selection_word_exist(misspel_df, unic_words)
word_count_accuracy_score = model_selection_word_count(misspel_df, unic_words)
tfidf_accuracy_score = model_selection_tfidf(misspel_df, unic_words)

/home/val/.local/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/val/.local/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/val/.local/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/val/.local/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/val/.local/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/val/.local/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: T

In [10]:
print(f"""Accuracy score by word exist: {word_exist_accuracy_score}
Accuracy score by word count: {word_count_accuracy_score}
Fccuracy score by tfidf: {tfidf_accuracy_score}""")

Accuracy score by word exist: 0.5170357751277683
Accuracy score by word count: 0.9522998296422487
Fccuracy score by tfidf: 0.5017035775127768
